In [1]:
import os
import sys
import git
import pathlib

import random
import time

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)
    
from libs import utils

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Extract model
# parser = argparse.ArgumentParser()
# parser.add_argument('model_instance')
# parser.add_argument('error_profile_tag')
# args= parser.parse_args()
model_instance = "mnist32-cnn_1024_256_64-1023"   #args.model_instance
error_profile_tag = "LIM_01-2188"                 #args.error_profile_tag

In [5]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)

test_images = test_images.astype(np.float32) / 255.0  
test_images = np.expand_dims(test_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
test_images = tf.image.resize(test_images, [image_x_size, image_y_size]) 

In [6]:
# Get model
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [7]:
# Load model
model = tf.keras.models.load_model(model_file)

In [8]:
# Load error profile
error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
error_profile_filename = error_profile_tag + ".npy"
error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
error_profile = np.load(error_profile_file)

In [9]:
# Batchsize for evaluation
EVAL_BATCHSIZE = 128

In [10]:
# Load test set
# Testing with only im_num images
im_num = 128*40#[128*78 = 9984]
test_set = (test_images[im_num:], test_labels[im_num:])

In [11]:
# No. of times to repeat experiments
K = 2

POP_SIZE = 3
N_GENERATIONS = 4

CR = 0.6
MR = 0.2

print("Population: ", POP_SIZE)
print("Generations: ", N_GENERATIONS)
print("Crossover Rate: ", CR)
print("Mutation Rate: ", MR)

# Run GA experiment


dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
# model_instance = dataset-modelarch_modelconfig-seed
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, 
                                                                           model_arch, 
                                                                           layer_widths, 
                                                                           seed)

# model_arch = 'cnn'
# model_config = '1024_256_64'
# model_type: 'mnist32-cnn_1024_256_64'
# model_meta_type: 'mnist32-cnn'

error_param = 3 # bitflips
err_type = "ERR_"+str(error_param)

# meta_optim = avg, ent
# quadrant = q1, q2, q3, q4
# dataset_seg_type = meta_optim +quadrant
dataset_seg_type = "all" # type of dataset segregation 

layer = "op"
EXP_TYPE = dataset_seg_type + "_" + model_meta_type + "_" + layer + "_" + err_type
experiment_name = model_instance + '--' + error_profile_tag

ga_experiment_name = dataset_seg_type+ "_" + experiment_name + '--ga_' + str(this_seed)
# Folder to save log files
logging_folder = pathlib.Path(PROJ_ROOT_PATH / "logging" / dataset_seg_type / model_meta_type / model_type / model_instance / err_type / EXP_TYPE)

if not os.path.exists(logging_folder):
    os.makedirs(logging_folder)
    
logging_filename_tag = pathlib.Path(logging_folder / ga_experiment_name)

from libs.ga.ga_experiments import EXP_mnist32_cnn_ERR_op
for i in range(K):
    print('#' * 20)
    print('Experiment number %d' % (i+1))
    # time_tag = time.strftime("%m%d_%H%M%S")
    logging_filename = str(logging_filename_tag)  + '_' + str(i) + '.csv'
    print("Logfile: ",logging_filename)

    experiment = EXP_mnist32_cnn_ERR_op(model=model,
                                        error_profile=error_profile,
                                        ERR_PARAM=error_param,
                                        test_set=test_set,
                                        log_file=logging_filename,
                                        pop_size=POP_SIZE,
                                        n_generations=N_GENERATIONS,
                                        crossover_rate=CR,
                                        mutation_rate=MR,
                                        experiment_tag=str(i))

    start = time.perf_counter()
    experiment.run()
    t = time.perf_counter() - start
    print('Experiment number %d: took %f(s)' % (i+1, t))

Population:  3
Generations:  4
Crossover Rate:  0.6
Mutation Rate:  0.2
####################
Experiment number 1
Logfile:  /repos/faulty_cnn/logging/all/mnist32-cnn/mnist32-cnn_1024_256_64/mnist32-cnn_1024_256_64-1023/ERR_3/all_mnist32-cnn_op_ERR_3/all_mnist32-cnn_1024_256_64-1023--LIM_01-2188--ga_84376_0.csv
Starting GA experiment...
GA: initializing first population
GA: start main loop...
GA: Generation 1: took 0.00(s)
	Best fitness: 0.993284
	No improvement for 1 gen(s)
GA: Generation 2: took 10.70(s)
	Best fitness: 0.993353
GA: Generation 3: took 8.84(s)
	Best fitness: 0.993421
GA: Generation 4: took 8.51(s)
	Best fitness: 0.993421
	No improvement for 1 gen(s)
Writing result to csv file...
Done
Experiment number 1: took 38.937397(s)
####################
Experiment number 2
Logfile:  /repos/faulty_cnn/logging/all/mnist32-cnn/mnist32-cnn_1024_256_64/mnist32-cnn_1024_256_64-1023/ERR_3/all_mnist32-cnn_op_ERR_3/all_mnist32-cnn_1024_256_64-1023--LIM_01-2188--ga_84376_1.csv
Starting GA ex